# 4. Основы синтасксиса: Функции и Структуры


## 4.1. Функции

Рассмотрим основной синтаксис для функций в Julia. Как уже отмечалось, команда `return` необязательна: функция в Julia возвращает результат последнего выражения, однако, мы будем придерживаться [`Blue CodeStyle`](https://github.com/JuliaDiff/BlueStyle)

In [578]:
# Основной синтаксис для функции
function r(x, y)
    return sqrt(x^2 + y^2)
end

# Краткий синтаксис
r_1(x, y) = sqrt(x^2 + y^2)

# Присвоение анонимной функции
r_2 = (x, y) -> sqrt(x^2 + y^2)


@show map(r, 3, 4) # Функция map применяет функцию r_3 с аргументами 3, 4
@show map(r_1, 3, 4)
@show map(r_2, 3, 4); 

map(r, 3, 4) = 5.0
map(r_1, 3, 4) = 5.0
map(r_2, 3, 4) = 5.0


Также для присвоения анонимной функции можно использовать конструкцию `begin-end`. В Julia не используются фигурные скобки, поэтому для любого выделения области видимости можно использовать `begin-end`.

In [579]:
# В несколько строк
r_4(x, y) = begin         
                z = r(x, y)
                return 2.0*z
            end
# Или в одну строчку
r_5(x, y) = begin z = r(x,y); return 2.0 * z end
@show map(r_4, 3, 4)
@show map(r_5, 3, 4); 

map(r_4, 3, 4) = 10.0
map(r_5, 3, 4) = 10.0


В теле функции может находиться несколько команд `return`, в этом случае при вызове сработает только одна из них.
Если необходимо, чтобы функция ничего не возвращала, тогда используется `return nothing`.
Тип `Nothing` имеет лишь одно значение `nothing`, представляющее в Julia отутствие значения.


In [580]:
function sayhi(name)
    println("Hi $name, it's great to see you!")
end

result = sayhi("John")

@show result
@show typeof(result);

Hi John, it's great to see you!
result = nothing
typeof(result) = Nothing


### Утиная типизация

*"Если она крякает как утка, это утка."* 

Функции Julia будут работать только с теми входными данными, которые имеют смысл. Например, теперь функция `say` работает над именем, записанным как целое число. 

А функция `f` будет работать и на матрице, а также будет работать со строкой типа "hi", потому что `*` определено для строковых входов как конкатенация строк.

In [581]:
f(x) = x^2
say(phrase) = "They said: $phrase"

@show say(55595472)
@show f(rand(2, 2))
@show f("hi");

say(55595472) = "They said: 55595472"
f(rand(2, 2)) = [0.08088585797803204 0.14588530617402243; 0.011231549573964415 0.056470987057099596]
f("hi") = "hihi"


### Mutating vs. non-mutating functions


По соглашению, функции, сопровождаемые `!`, изменяют свое содержимое, а функции без `!`, не делают этого. 

Например, давайте посмотрим на разницу между `sort` и` sort! `.

In [582]:

vector = [3, 5, 2]
@show vector

@show sort(vector; rev = true) # сортировка элементов vector
@show vector

@show sort!(vector; rev = true) # сортировка элементов с изменением vector
@show vector;

vector = [3, 5, 2]
sort(vector; rev = true) = [5, 3, 2]
vector = [3, 5, 2]
sort!(vector; rev = true) = [5, 3, 2]
vector = [5, 3, 2]


### Векторизация функций


В Julia *любая функция* может быть векторизована. Т.е. вы можете создать функцию для скаляров и автоматически получаете её векторизованный аналог. Векторизация осуществляется через механизм *broadcasting*. При этом достаточно в конце имени функции дописать точку `.` (*dot-syntax*).


In [584]:
@show x = [0, π/6, π/3]
@show sin.(x);

x = [0, π / 6, π / 3] = [0.0, 0.5235987755982988, 1.0471975511965976]
sin.(x) = [0.0, 0.49999999999999994, 0.8660254037844386]


Механизм векторизации «насыщен» (fused). Для программиста это означает, что сложный вызов, например, sin.(cos.(x)) не медленнее цикла, в котором к каждому элементу вектора x применили sin(cos(x[i])).

Автоматически векторизуются и пользовательские функции (см. дальше) над стандартными типами.


In [2]:
f(x) = x^2 + 1
@show f.([1, 2, 3])

g(x, y) = f(x) + f(y)
@show g.([1, 2], [3, 4])
@show g.([1, 2], 3);


f.([1, 2, 3]) = [2, 5, 10]
g.([1, 2], [3, 4]) = [12, 22]
g.([1, 2], 3) = [12, 15]


Операторы тоже функции. Так, broadcast порождает, например, поэлементные операции над векторами и матрицами (`.+`, `.*`, etc...).

In [586]:
@show 2 * sin.(x) .* cos.(x)
@show sin.(2 * x)

(2 * sin.(x)) .* cos.(x) = [0.0, 0.8660254037844386, 0.8660254037844388]
sin.(2x) = [0.0, 0.8660254037844386, 0.8660254037844387]


3-element Vector{Float64}:
 0.0
 0.8660254037844386
 0.8660254037844387

*Полиморфизм*

В этом примере несколько поэлементых видов умножений.

- `2 * x`: умножение скаляра на вектор, оно и без броадкаста действует поэлементно;
- `x .* y`: поэлементное умножение векторов.

Если вам надоедает ставить точки везде, воспользуйтесь макросом `@.`


In [587]:
z = @. 2 * sin(x) * cos(x)
@show z;

z = [0.0, 0.8660254037844386, 0.8660254037844388]


## 4.2. Типы в Julia

В Julia типы упорядочены в *дерево*, образуя систему.
В этом дереве есть самые разные ветви: для числовых типов, строковых, массивов...
В свою очередь, для функции можно написать любое количество *методов*, указывая в объявлении метода при каком наборе типов аргументов, или даже целых ветвей, он будет вызываться.
Процесс выбора метода при вызове функции называется *диспетчеризацией*.
Диспетчеризация учитывает типы *всех позиционных аргументов*, поэтому она называется *множественной*.
При первом вызове функции или метода Julia, если возможно, создаёт для него эффективную (машинную) реализацию.

Эти три инструмента: дерево типов, методы и диспетчеризация, являются ядром дизайна языка Julia, дающим гибкость в написании кода вообще и быстрого кода в частности.

Подробнее о рассматриваемых здесь темах можно почитать в разделах [Types](https://docs.julialang.org/en/v1/manual/types/), [Methods](https://docs.julialang.org/en/v1/manual/methods/) и [wiki:Multiple dispatch](https://en.wikipedia.org/wiki/Multiple_dispatch#Languages_with_built-in_multiple_dispatch).


### Система типов

В Julia **сильная динамическая** система типов.

Однако, стоит избегать и потенциально предугадывать места, в которых переменная меняет свой тип. Компилятор вас поощрит.

Поскольку в Julia динамическая типизация, то переменная может менять свой тип в процессе работы программы.

In [588]:
x = 5
@show typeof(x)
x = 5.0
@show typeof(x)

typeof(x) = Int64
typeof(x) = Float64


Float64

В Julia нельзя указать, чтобы переменная имела постоянное значение, но можно указать, что переменная *не меняет свой тип*:

In [589]:
const y = 10
@debug y = 10.0 # ERROR: invalid redefinition of constant y
y = 20;

Julia не допустила присвоение `y = 10.0`, потому что оно меняет тип переменной `y::Int64`. Однако, присваивать значения `Int64` переменной `y` всё-таки можно, но не стоит этого делать.

### Декларация типов

С помощью оператора `::` вы можете *декларировать* тип объекта. Обычно это делается для

- переменных;
- аргументов функции;
- возвращаемого функцией значения;
- полей композитных типов (структур).


> `NOTE` Конвертацию делает функция `convert(T, x)`. Правила конвертации можно задавать.

Например, вы можете декларировать тип переменной внутри функции (или другой локальной области видимости). В таком случае при присвоении значения происходит конвертация.

In [590]:
function foo(x::Int64)
    return x
end

function bar(x)::Int8
    return x
end

@show typeof(foo(10))
@show typeof(bar(10))

typeof(foo(10)) = Int64
typeof(bar(10)) = Int8


Int8

### Какие бывают типы

Типы в Julia классифицируются по следующим признакам (указаны не все)

- абстрактный *abstract* (`AbstractFloat`) и конкретный *concrete* (`Float64`);
- примитивный *primitive* (`Float64`) и композитный *composite* (`Complex{Float64}`);
- параметрический *parametric* (`Complex{Float64}`);
- изменяемый (`Vector{Float64}`) и неизменяемый (`Tuple{Float64,Float64}`);
- ...

В Julia система типов организована в дерево, корень которого тип `Any`, т.е. любой.
Листьями дерева типов являются **конкретные типы**.
У значения такого типа известна структура в памяти.
**Абстрактные** типы нужны в качестве промежуточных узлов дерева типов, выстраивая иерархию.

>`NOTE` Кроме того, объявление неабстрактного параметрического типа `T1{T2}`  создаёт **UnionAll** тип `T1`. > Последний ведёт себя как супертип (родительский) для параметризованных потомков.
>
>Например, `Rational{T}` порождает UnionAll `Rational`, и `Rational` будет супертипом для всех `Rational{T}`: `Rational{Int64}`, `Rational{Int8}`...
>
>В дереве ниже `UnionAll` типы помещены в параллелограммы.

Например, так выглядит часть дерева с числовыми и строчными типами, при этом

- конкретные типы указаны в округлённых рамках;
- абстрактные в прямоугольных.


In [591]:
"""
Any
├─AbstractString 
│  ├─String
│  └─SubString
└─Number
   ├─ Complex
   └─ Real
      ├─ AbstractFloat
      │  └─ ...
      ├─ Integer
      │  ├─ Bool
      │  ├─ Signed
      │  │  ├─ Int8
      │  │  ├─ Int16
      │  │  ├─ Int32
      │  │  ├─ Int64
      │  │  ├─ Int128
      │  │  └─ BigInt
      │  └─ Unsigned
      │     └─ ...
      ├─ Rational
      │  └─ ...
      └─ AbstractIrrational
         └─ Irrational
""";

Несколько функций для интроспекции системы типов

- `subtypes(T)`: подтипы типа `T`;
- `isabstracttype(T)`: является ли тип `T` абстрактным;
- `isprimitivetype(T)`: является ли тип `T` примитивным;
- `ismutable(x)`: является ли значение `x` изменяемым.

```{margin}
С точки зрения дерева, можно ли из `Ty` добраться до `Tx`.
```
Также с помощью subtype-оператора `Tx <: Ty` можно проверять "является ли тип `Tx` подтипом типа `Ty`".

In [592]:
@show Int64 <: Number;

Int64 <: Number = true


### Композитные типы

**Композитный** тип имеет более сложную структуру в памяти, чем примитивный тип.
Этот тип является набором именованных полей, а экземпляром такого типа можно манипулирвовать как одним значением.
В других языках им соответствуют объекты (*objects*) или структуры (*structs*).
Примерами встроенных композитых типов являются `Complex{T}`, `Rational{T}`, `Tuple`, `String`, `Array`, `IO` etc...

Ниже показан пример композитного типа для точки на плоскости.

По умолчанию, структуры в Julia неизменяемые.
Перезапись поля можно разрешить, сделав структуру изменяемой (`mutable`).

In [593]:
struct Point
    x        # то же, что и x::Any, т.е. поле может быть либого типа
    y::Int   # так указывается тип (можно и абстрактный)
end

p1 = Point(1.0, 2)
@show typeof(p1)
@show p1
@show p1.x
@debug p1.x = 9

mutable struct MPoint
    x
    y
end

mp1 = MPoint(1, 2)
@show mp1
mp1.x = 10
@show mp1;

#Объявления абстрактного типа
abstract type AbstractPoint <: Any end



typeof(p1) = Point
p1 = Point(1.0, 2)
p1.x = 1.0
mp1 = MPoint(1, 2)
mp1 = MPoint(10, 2)



Иногда это может приводить к замедлению работы, поскольку `mutable struct` (или её часть) выделяется в куче, а не на стеке.

Тем не менее, в поле обычной структуры можно хранить значение изменяемого типа.
В таком случае поле хранит *ссылку на изменяемый объект*.
Объект поменять можно, а ссылку &ndash; нет.
Например, так хранятся массивы внутри структур.

Конструктор (*constructor*) `Point(x, y)` для композитного типа можно поменять или создать несколько.
Конструкторы бывают внутренние (*inner constructor*) и внешние (*outer constructor*).
Внутренние конструкторы объявляются внутри объявления структуры, и в этом случае конструктор по умолчанию теряется.
Внешние конструкторы, по сути, функции для удобного создания экземпляров структур.
Обычно такие функции имеют то же имя, что и тип.
Такое возможно благодаря механизму методов, о котором говорится далее.

### Параметрические композитные типы

Параметрический тип это тип, который в своём объявлении содержит дополнительную информацию. Например, тип `Complex` в языке объявлен следующим образом:

In [594]:
struct Complex{T<:Real} <: Number
    re::T
    im::T
end

С практической точки зрения, код выше объявляет структуру данных `Complex` для комплексных чисел, которые (сюрприз) являются числами (`Number`).
Т.е., подразумевается, что с ними можно совершать все арифметические операции.
Сама структура данных состоит из двух действительных (`Real`) чисел с *одинаковым* типом.

Более строго, объявление выше значит

```{margin}
`Complex` в данном случае ни абстрактный, ни конкретный.
```
- создать UnionAll-тип `Complex`, который будет вести себя как супертип для `Complex{T}`;
- создать параметрический тип `Complex{T}`,
- где параметр `T` является подтипом типа `Real`,
- при этом типы `Complex` и `Complex{T}` являются подтипами `Number`;
- у `Complex{T}` два поля: `re` и `im`, каждое из них имеет тип `T`.

```{margin}
Это не значит, что массив в Julia может хранить только значения одного типа. Ведь есть абстрактные типы: `Any`, `Number`...
```
В качестве параметров могут быть типы и значения примитивных типов. Например, в Julia объявлен тип для массивов ` AbstractArray{T,N}`, где под `T` подразумевается тип значений, а под `N` размерность массива (1 для векторов, 2 для матриц...).

Параметрические типы порождают целое семейство типов. Член такого семейства &ndash; *любая  комбинация разрешенных значений* параметров типа.

Таким образом, с помощью параметризации композитного типа мы можем дать подсказки для компилятора, чтобы тот создавал оптимизированный код.


### Union типы

Тип можно рассматривать как набор возможных значений. Тип выражает *неопределенность* относительно того, какое значение мы имеем. Вы можете делать операции над ними. Типы Union, естественно, обобщают отсутствующие данные.

In [595]:
struct IFPoint 
    x::Union{Integer,AbstractFloat}
    y::Union{Integer,AbstractFloat}
end

p1 = IFPoint(1, 2.0)
@show p1

@show Union{Int,String} <: Union{Int,String,Float32}

data = [1.1, missing, 3.2, missing, 5.7, 0.4]
@show typeof(data);

p1 = IFPoint(1, 2.0)
Union{Int, String} <: Union{Int, String, Float32} = true
typeof(data) = Vector{Union{Missing, Float64}}


`Array{T,1} where T<:Integer`

означает «объединение всех типов вида Array {T, 1}, где T является подтипом Integer».

Это выражает неопределенность в отношении значения параметра.

Эта концепция существует во всех версиях Julia, но не имеет синтаксиса или полностью правильной поддержки в системе до следующего v0.6.0 (в настоящее время в ветке jb / subtype).

* Поскольку этот тип вводит *переменные*, его выразительная сила (вероятно) эквивалентна количественным логическим формулам.
* Требует  q-SAT решатель в компиляторе.
* По общепринятым предположениям, сложнее, чем NP-полная.

In [596]:
# это определение в базовой библиотеке
MyVector = Array{T,1} where T

# Функция возвращает тип, принадлежащий Integer
func(a::T) where {T<:Integer} = T
func(0x00)

UInt8

## 4.3 Multiple dispatch

В этом блокноте мы рассмотрим **множественную диспетчеризацию (мультиметоды)**, которая является ключевой особенностью Julia. Мультиметоды делают программное обеспечение *универсальным* и *быстрым*.

Для каждой функции в Julia можно определить сколько угодно методов (*methods*). Это способ полиформизма в языке. Вы уже с ним сталкивались, например, `1 + 2` и `1.0 + 2.0` совершенно разные вызовы. В первом случае вызывается метод `+(x, y)` для  `Int64`, а во втором метод `+(x, y)` для сложения `Float64`.

Вообще, у сложения `+` в базовой библиотеке Julia 1.6.2 190 методов.
Здесь методы и для примитивных типов, для комплексных и рациональных чисел, для векторов, матриц, тензоров...

Узнать, какой метод вызвался можно макросом `@which`


In [597]:
methods_of_plus = methods(+)
@show length(methods_of_plus)
for k=1:5
    @show methods_of_plus[k]
end

method_1 = @which 1 + 2
method_2 = @which 1.0 + 2.0

@show method_1
@show method_2;

length(methods_of_plus) = 195
methods_of_plus[k] = +(z::Base.Complex{Bool}, x::Bool) @ Base complex.jl:306
methods_of_plus[k] = +(z::Base.Complex{Bool}, x::Real) @ Base complex.jl:320
methods_of_plus[k] = +(z::Base.Complex, x::Bool) @ Base complex.jl:313
methods_of_plus[k] = +(x::Bool, z::Base.Complex{Bool}) @ Base complex.jl:305
methods_of_plus[k] = +(x::Bool, y::Bool) @ Base bool.jl:166
method_1 = +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} @ Base int.jl:87
method_2 = +(x::T, y::T) where T<:Union{Float16, Float32, Float64} @ Base float.jl:409


Можно сказать, что функция определяет дефолтное действие с аргументами. А методы функции содержат частные реализации этих действий на случаи тех или иных аргументов. Например, метод `sum` для вектора сложит все элементы и это займёт линейное время, а `sum` для арифметической прогрессии займёт константное время, используя формулу. При этом с точки зрения пользователя сигнатура вызова одна и та же: `sum(A)`.

Процесс выбора нужного метода называется **диспетчеризацией** (*dispatch*). В Julia диспетчеризация производится **по типам *всех* позиционных аргументов функции**. Этот механизм называется *multiple dispatch*. Он гибче и продуктивнее, нежели диспетчеризация по одному типу, например, как в Python (`type(x).__add__(x, y)`). При этом, в отличие от языков со статической типизацией, нет необходимости указывать типы аргументов.

Строго говоря, *generic function* единственна, а методов у неё много. Однако, всё же принято называть методы функциями, если контекст позволяет. Методы создаются как функции, но с указанием типа хотя бы одного аргумента. 

При диспетчеризации может возникать коллизия: два или более метода подходят для вызова.
В этом случае Julia выдаст ошибку `MethodError`.


In [598]:
g(x, y) = 2x + y
g(x::Float64, y::Float64) = 3x + y
g(x::Complex, y::Complex) = 5x + y*im

for m in methods(g)
    println(m)
end

g(x::Float64, y::Float64) @ Main ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:2
g(x::Real) @ Main ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:1
g(x::Complex, y::Complex) @ Main ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:3
g(x, y) @ Main ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:1


### Такая разная диспетчеризация

Базовая диспетчеризация

In [599]:
fee(a, b) = "fallback"
fee(a::Number, b::Number) = "a and b are both numbers"
fee(a::Number, b) = "a is a number"
fee(a, b::Number) = "b is a number"
fee(a::Integer, b::Integer) = "a and b are both integers"
println(methods(fee))
@show fee(1.5, 2)
@show fee(1, "bar")
@show fee(1, 2)
@show fee("foo", [1,2]);

# 5 methods for generic function "fee" from Main:
 [1] fee(a::Integer, b::Integer)
     @ ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:5
 [2] fee(a::Number, b::Number)
     @ ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:2
 [3] fee(a::Number, b)
     @ ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:3
 [4] fee(a, b::Number)
     @ ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:4
 [5] fee(a, b)
     @ ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:1
fee(1.5, 2) = "a and b are both numbers"
fee(1, "bar") = "a is a number"
fee(1, 2) = "a and b are both integers"
fee("foo", [1, 2]) = "fallback"


"Диагональная" диспетчеризация

In [600]:
d(a::T, b::T) where {T<:Number} = "a and b are both $(T)s"
d(a::T, b::T) where {T<:Integer} = "both are $T integers"
println(methods(d))
@show d(big(1.5), big(2.5))
@show d(big(1), big(2)); 

# 2 methods for generic function "d" from Main:
 [1] d(a::T, b::T) where T<:Integer
     @ ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:2
 [2] d(a::T, b::T) where T<:Number
     @ ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:1
d(big(1.5), big(2.5)) = "a and b are both BigFloats"
d(big(1), big(2)) = "both are BigInt integers"


Методы с неполным списком аргументов

In [601]:
φ(args::Number...) = "$(length(args))-ary heterogeneous call"
φ(args::T...) where {T<:Number} = "$(length(args))-ary homogeneous call"
@show φ(1)
@show φ(1, 2, 3)
@show φ(1, 1.5, 2)
@show φ()
@show φ(1, 2)
@show φ([1, 2, 3]...);

φ(1) = "1-ary homogeneous call"
φ(1, 2, 3) = "3-ary homogeneous call"
φ(1, 1.5, 2) = "3-ary heterogeneous call"
φ() = "0-ary homogeneous call"
φ(1, 2) = "2-ary homogeneous call"
φ([1, 2, 3]...) = "3-ary homogeneous call"


Необязательные аргументы

In [602]:
χ(x, y = 0) = 2x + 3y
println(methods(χ))

# 2 methods for generic function "χ" from Main:
 [1] χ(x)
     @ ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:1
 [2] χ(x, y)
     @ ~/Рабочий стол/Кодирование/Julia/handbook-julia/5.functions.ipynb:1


### Ключевые слова

Аргументы перечисляемые после точки с запятой в прототипе функции являются ключевыми словами, к которым можно непосредственно обращаться при вызове функции

In [603]:
k(x, y = 0; opt::Bool = false) = opt ? 2x+y : x+2y
@show k(2)
@show k(2, 3)
@show k(2, opt=true)
@show k(2, 3, opt=true);


k(2) = 2
k(2, 3) = 8
k(2, opt = true) = 4
k(2, 3, opt = true) = 7


Суммируя наши знания, можем написать функцию, которая парсит неопределенное количество необязательных аргументов. 

In [604]:
a, b = 3, 4
function allkw(; kw...)
    for (k, v) in kw
        println("$k = $v")
    end
end
allkw(a=1,b=2);

a = 1
b = 2


### Стабильность по типу

Когда Julia исполняет функцию (или метод) в первый раз, компилятор "на лету" создаёт машинный код и запоминает его для последующих вызовов уже без компиляции.
Получившийся код лишь может быть по-настоящему машинно-эффективным (как в компилирующихся языках, например, в C/C++ или Fortran).
Например, если в вашей функции содержится переменная, тип которой можно узнать _только при исполнении_ (runtime), то эффективной реализации исходного кода, касающегося работы с этой переменной, не получится.
В этом случае какую-либо операцию с такой переменной необходимо проверять: существует ли она и, если существует, то где определена.
С другой стороны, если по типам аргументов _при вызове_ функции компилятору понятно, какие типы у всех остальных переменных, включая возвращаемое значение, то созданный машинный код будет эффективным.

В итоге, программы на Julia могут быть быстрыми, как в компилируемых языках, но могут быть и медленными, как в интерпретируемых языках.
Чтобы писать быстродействующие программы, стоит разбивать исходный код на небольшие функции, а в каждой из них поддерживать _стабильность типов_.

>`DEFINITION` Стабильная по типу функция
>
>Функция, тип возвращаемого значения которой зависит только от типов аргументов, и не зависит от их значений, называют стабильной по типу (type stable function).

Рассмотрим для примера функцию `f(x, y) = x^2 + y^2`.

При первом вызове от двух `Int` (например, `f(3, 4)`) компилятор удачно выводит типы всех значений внутри функции: `x^2 == 9` и `y^2 == 16` это два `Int`, а их сумма `25` (возвращаемое значение) также `Int`.
Более того, при любых вызовах вида `f(::Int, ::Int)` промежуточные значения и возвращаемое значение будут типа `Int`.
Поэтому можно создать машинную реализацию для вызова `f(::Int, ::Int)` и использовать её в дальнейшем.
Аналогично компилятор поступит для вызовов вида `f(::Float64, ::Int)`, `f(::Int, ::Float64)` и `f(::Float64, ::Float64)`.

В нашем примере стабильные по типу функции это `^(x, y)`, `+(x, y)` и сама `f`.
При этом стабильность по типу функции `f` в нашем случае обеспечивается стабильностью функций `+` и `^`.
Для таких функций компилятору известно устройство в памяти входных и выходных значений, что позволяет создавать эффективный код.

Покажем также пример нестабильной по типу функции:


In [605]:
h(x::Real) = x > 0 ? x : 0

h (generic function with 3 methods)

В этом случае при вызове `g(::Int)` функция стабильна по типу, поскольку для любых `x::Int` возвращется `Int`.
Однако, вызов вида `g(::Float64)` не стабилен по типу, поскольку для положительных `x::Float64` возвращается число типа `Float64`, а для отрицательных --- число типа `Int`.
Добиться стабильности для вызова `g(::Float64)` можно было бы добавлением метода `g(x::Float64) = x > 0 ? x : 0.0`, но тогда подобные методы требовались бы для всех типов действительных чисел `T <: Real`.
Лучшим вариантом является использование функции-утилиты `zero(x)`: она возращает ноль того же типа, что и `x`.

In [606]:
h1(x::Real) = x > 0 : x : zero(x)

h1 (generic function with 1 method)

Также рекомендуется заменить `x > 0` на `x > zero(x)`, но компилятор, вероятно, сгенерирует идентичный машинный код (проверьте с помощью `@code_native`).

Показательным примером дизайна стабильной по типу функции в Julia является дизайн функции квадратного корня `sqrt(x)`.
Если извлекать квадратный корень из натуральных чисел, то только для некоторых из них (полных квадратов) получим натуральное число, а в остальных случаях получим не целые числа.
Разработчики Julia учли эту особенность, и, ради сохранения стабильности, вызов `sqrt(::Int)` всегда возвращает `Float64`, даже для полных квадратов.
Кроме того, если извлекать корень из отрицательного числа, то получим число комплексное.
Поэтому `sqrt(x::Real)` требует неотрицательного числа, чтобы независимо от значения `x` возвращать `Float64`.
А для извлечения корней из отрицательных чисел требуется использовать метод вида `sqrt(x::Complex)` (например, `sqrt(-1 + 0im)` вместо `sqrt(-1)`).
В последнем случае всегда возвращается `Complex` число, что сохраняет стабильность по типу функции `sqrt`.

Для создания стабильных по типу функций Julia предоставляет множество инструментов.
Например,
- `zero(x)` и `one(x)` для создания нуля и единицы того же типа, что и `x`,
- `similar(x)` для создания массива того же типа и размера, что и `x`,
- `iszero(x)` и `isone(x)` вместо `x == 0` и `x == 1`,
- `isodd(x)` и `iseven(x)` вместо `x % 2 == 1` и `x % 2 == 0`,
- `isnan(x)` и `isinf(x)` для проверки на Not-a-Number и бесконечность.

Часто используется функция `promote(x...)`, которая подбирает "общий" тип для своих аргументов и приводит их к нему. А для проверки своего кода на стабильность по типу можно использовать макрос `@code_warntype`.


In [3]:
promote(1, 2.0, 3//4)

(1.0, 2.0, 0.75)

### Интерфейсы

На методах основаны интерфейсы в языке.
Например, для типа точки на декартовой плоскости, рассмотренном выше, вы можете определить оператор сложения.
Вы также можете создать вектора из точек.
Сложение таких векторов определять уже не нужно: операция сложения для векторов происходит поэлементно и определено в стандартной библиотеке.
Поэтому, определив арифметические операции для точки, вам автоматически доступны методы линейной алгебры (`LinearAlgebra`) и, например, статистики (`Statistics`).

Вы можете создавать собственные структуры данных, у которых будет поведение коллекций.
Как только вы объявите необходимые методы интерфейса для своего типа, вы сможете пользоваться всеми функциями, определенными для коллекций.
Или, скажем, вы можете встроить свои структуры данных в интерфейс цикла `for`.

С помощью методов также создаются типы, ведущие себя как функции: их можно вызывать (*callable*).
На этом основаны замыкания (*closures*) в Julia.

Один из ярких примеров это задание типа полиномиальной функции, рассмотренного ниже:

In [613]:
# Создание структуры коэффициентов полинома
struct Polynomial{R}
    coeffs::Vector{R}
end

# Объявление "вызова" для типа Polynomial
function (p::Polynomial)(x)
    v = p.coeffs[end]
    for i = (length(p.coeffs)-1):-1:1
        v = v*x + p.coeffs[i]
    end
    return v
end

sign(x) = (x ≥ 0) ? "+" : "-"

# Функция для отображения Polynomial в командной строке
Base.show(io::IO, p::Polynomial) = begin
    v = p.coeffs[end]
    for i = (length(p.coeffs)):-1:1
        if p.coeffs[i]==0 
            continue 
        else
            if i==1 
                print(io, " $(sign(p.coeffs[i])) $(abs(p.coeffs[i]))") 
            else 
                if i!=length(p.coeffs)
                    print(io, " $(sign(p.coeffs[i])) ")
                end
                if p.coeffs[i]==1
                    print(io, "x^$(i-1)") 
                else
                    print(io, "$(abs(p.coeffs[i]))x^$(i-1)") 
                end
            end 
        end
    end
end

Base.ndims(p::Polynomial) = length(p.coeffs) - 1

# Задание случая отсутствия аргументов функции
(p::Polynomial)() = p(0)

# Функции для коэффициентов суммы полиномов
function psum(a,b)
    z = (length(a)>length(b)) ? copy(a) : copy(b)
    for k=1:min(length(a), length(b))
        z[k] = a[k] + b[k]
    end
    return z
end

# Функции для коэффициентов произведения полиномов
function pmul(a, b)
    len = length(a)+length(b)-1
    a1 = psum(a, zeros(Int64, len))
    b1 = psum(b, zeros(Int64, len))
    z = zeros(Int64, len)
    for k=1:len
        for i=1:k
            z[k] += a1[i]*b1[k-i+1]
        end
    end
    return z 
end

# Объявление сложения, умножения и возведения в степень полиномов
import Base: *, +, ^
+(f::Polynomial, g::Polynomial) = Polynomial(psum(f.coeffs, g.coeffs)) # Cложение
+(α::Number, g::Polynomial) = Polynomial(psum([α], g.coeffs)) # Cложение
*(f::Polynomial, g::Polynomial) = Polynomial(pmul(f.coeffs, g.coeffs)) # Умножение
*(α::Number, g::Polynomial) = Polynomial(α.*g.coeffs) # Умножение
^(f::Function, n::Integer) = n == 1 ? f : f*f^(n-1)

# Объявление полиномиальной функции
p = Polynomial([1,10,100])
x = Polynomial([0,1])
@show p
@show ndims(p)
@show p(3)
p1 = x^2
p2 = x - 1
@show 2*p1 - p2 + 3
@show p2^5;

# Экспонента через ряд Тейлора
pow(n) = Polynomial([ i!=(n+1) ? 0 : 1 for i=1:(n+1)])
myexp = sum(1//factorial(big(n)) * pow(n) for n in 0:100);  
[myexp(1); exp(big(1))];

p = 100x^2 + 10x^1 + 1
ndims(p) = 2
p(3) = 931
(2p1 - p2) + 3 = 2x^2 - 1x^1 + 4
p2 ^ 5 = x^5 - 5x^4 + 10x^3 - 10x^2 + 5x^1 - 1
